Libraries and Style Files
-------------------------

In [1]:
#we may need some code in the ../python directory and/or matplotlib styles
import sys
sys.path.append('../python/')

#matplotlib for plotting
import matplotlib as mpl
from matplotlib import pyplot as plt
plt.style.use('../mplstyles/stylelib/standard.mplstyle')

import numpy as np

Isotopic Abundances
===================



In [ ]:
import ENDF6el as el
import flux_recoil_spec as frs
import time
import importlib

#get Er vector
Er= np.geomspace(100/1e6, 1.0, 100)
print(np.shape(Er))
E,F,ff,ffspec = frs.SNOLAB_flux(Enmin=1e-3)
start = time.time()
drde,dsig=frs.dRdErfast(Er,E,F,N=1)
end = time.time()
print('Evaluation Time: {:1.5f} sec.'.format(end-start))